In [18]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from keras import backend as K
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
K.set_session(sess)

from keras.layers import Input, Dense, Flatten, GlobalAveragePooling2D, Activation, Conv2D, MaxPooling2D, BatchNormalization, Lambda
from keras.models import Model, load_model, model_from_json
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix
import matplotlib.pyplot as plt
from scipy.optimize import brentq
from scipy.interpolate import interp1d
import glob
from PIL import Image
from tqdm import tqdm, trange

In [19]:
nb_classes = 1  # number of classes
img_width, img_height = 224, 224  # change based on the shape/structure of your images
batch_size = 32  # try 4, 8, 16, 32, 64, 128, 256 dependent on CPU/GPU memory capacity (powers of 2 values).
nb_epoch = 50  # number of iteration the algorithm gets trained.
learn_rate = 1e-5  # sgd learning rate

## DIR ##

In [20]:
train_dir = '/home/skkulab/ICCV/Dataset/preprocessed_dataset/train'
validation_dir = '/home/skkulab/ICCV/Dataset/preprocessed_dataset/validation'
test50_dir = '/home/skkulab/ICCV/Dataset/preprocessed_dataset/test50'
test75_dir = '/home/skkulab/ICCV/Dataset/preprocessed_dataset/test75'
test95_dir = '/home/skkulab/ICCV/Dataset/preprocessed_dataset/test95'
test99_dir = '/home/skkulab/ICCV/Dataset/preprocessed_dataset/test99'

# AlexNet

In [21]:
img_input = Input(shape=(img_height, img_width, 3))

x = Conv2D(96, 11, strides=4, padding='same', use_bias=False)(img_input) # 15
x = Activation('relu')(x)

x = Conv2D(256, 5, strides=1, padding='same', use_bias=False)(x)
x = Activation('relu')(x)

x = MaxPooling2D(pool_size=3, strides=2, padding='valid')(x) # 8

x = Conv2D(384, 3, strides=1, padding='same', use_bias=False)(x) # 15
x = Activation('relu')(x)

x = MaxPooling2D(pool_size=3, strides=2, padding='valid')(x) # 8

x = Conv2D(384, 3, strides=1, padding='same', use_bias=False)(x) # 15
x = Activation('relu')(x)
x = Conv2D(256, 3, strides=1, padding='same', use_bias=False)(x)
x = Activation('relu')(x)

model_out = MaxPooling2D(pool_size=3, strides=2, padding='valid')(x) # 8
# Add fully connected layer
x = GlobalAveragePooling2D()(model_out)
x = Dense(4096, activation=None)(x)
x = Activation('relu')(x)
x = Dense(1, activation=None)(x)
out = Activation('sigmoid')(x)

model = Model(img_input, out)
print(model.summary())
print(len(model.trainable_weights))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 56, 56, 96)        34848     
_________________________________________________________________
activation_8 (Activation)    (None, 56, 56, 96)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 56, 56, 256)       614400    
_________________________________________________________________
activation_9 (Activation)    (None, 56, 56, 256)       0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 27, 27, 256)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 27, 27, 384)       884736    
__________

In [22]:
model.compile(optimizer=Adam(lr=learn_rate),
              loss='binary_crossentropy',
              metrics=['accuracy'])

print(len(model.trainable_weights))

9


## Data generator

In [70]:
def generator(directory, batch_size=32):
    folder =  np.sort(os.listdir(directory))
    real_img = np.asarray(glob.glob(directory + '/' + folder[0]+'/*.png'))
    real_idx = np.arange(len(real_img))
    fake_img = np.asarray(glob.glob(directory + '/' + folder[1]+'/*.png'))
    fake_idx = np.arange(len(fake_img))
    
    while 1:
        X0 = []
        X1 = []
        X2 = []
        y = []
        
        if (len(real_idx) < batch_size*2):
            real_idx = np.arange(len(real_img))
            continue
        
        if (len(fake_idx) < batch_size*2):
            fake_idx = np.arange(len(fake_img))
            continue
        
        for _ in range(batch_size):
            if (len(real_idx) < batch_size*2):
                real_idx = np.arange(len(real_img))
                break
            if (len(fake_idx) < batch_size*2):
                fake_idx = np.arange(len(fake_img))
                break
                
            anchor_choice = np.random.randint(2)
            anchor_idx = np.random.choice(real_idx if anchor_choice == 0 else fake_idx, 1, replace=False)
            if anchor_choice == 0:
                real_idx = real_idx[~np.isin(real_idx, anchor_idx)]
                X0.append(np.asarray(Image.open(real_img[anchor_idx[0]]).convert("RGB"))/255.)
                random1 = np.random.choice(real_idx, 1, replace=False)
                real_idx = real_idx[~np.isin(real_idx, random1)]
                random2 = np.random.choice(fake_idx, 1, replace=False)
                real_idx = fake_idx[~np.isin(fake_idx, random2)]

                X1.append(np.asarray(Image.open(real_img[random1[0]]).convert("RGB"))/255.)
                X2.append(np.asarray(Image.open(fake_img[random2[0]]).convert("RGB"))/255.)
                y.append(np.array([anchor_choice]))
            else:
                fake_idx = fake_idx[~np.isin(fake_idx, anchor_idx)]
                X0.append(np.asarray(Image.open(fake_img[anchor_idx[0]]).convert("RGB"))/255.)
                random1 = np.random.choice(real_idx, 1, replace=False)
                real_idx = real_idx[~np.isin(real_idx, random1)]
                random2 = np.random.choice(fake_idx, 1, replace=False)
                real_idx = fake_idx[~np.isin(fake_idx, random2)]

                X2.append(np.asarray(Image.open(real_img[random1[0]]).convert("RGB"))/255.)
                X1.append(np.asarray(Image.open(fake_img[random2[0]]).convert("RGB"))/255.)
                y.append(np.array([anchor_choice]))
            
        X0 = np.asarray(X0)
        X1 = np.asarray(X1)
        X2 = np.asarray(X2)
        y = np.asarray(y)
        yield [X0, X1, X2], y
        
def generator_res(ft_dir, directory, batch_size=1, critical_value=0.5):
    folder = np.sort(os.listdir(directory))
    ft_real_img = np.asarray(glob.glob(ft_dir + '/' + '0' +'/*.png'))
    ft_fake_img = np.asarray(glob.glob(ft_dir + '/' + '1' + '/*.png'))
    ft_real_idx = np.arange(len(ft_real_img))
    ft_fake_idx = np.arange(len(ft_fake_img))
    fake_img = np.asarray(glob.glob(directory + '/' + folder[1] + '/*.png'))
    fake_idx = np.arange(len(fake_img))
    real_img = np.asarray(glob.glob(directory + '/' + folder[0] + '/*.png'))
    real_idx = np.arange(len(real_img))
    while 1:
        X0 = []
        X1 = []
        X2 = []
        y = []
        if (len(fake_idx) < batch_size):
            break
        if (len(real_idx) < batch_size):
            break
        if (len(ft_real_idx) < batch_size):
            break
        if (len(ft_fake_idx) < batch_size):
            break
        for _ in range(batch_size):
            anchor_choice = np.random.randint(2)
            anchor_idx = np.random.choice(ft_real_idx if anchor_choice == 0 else ft_fake_idx, 1, replace=False)
            if anchor_idx == 0:
                if (len(ft_real_idx) < batch_size):
                    break
                if (len(real_idx) < batch_size):
                    break
                if (len(fake_idx) < batch_size):
                    break
                ft_real_idx = ft_real_idx[~np.isin(ft_real_idx, anchor_idx)]
                X0.append(np.asarray(Image.open(ft_real_img[anchor_idx[0]]).convert("RGB"))/255.)
                random1 = np.random.choice(real_idx, 1, replace=False)
                real_idx = real_idx[~np.isin(real_idx, random1)]
                random2 = np.random.choice(fake_idx, 1, replace=False)
                fake_idx = fake_idx[~np.isin(fake_idx, random2)]
                X1.append(np.asarray(Image.open(real_img[random1[0]]).convert("RGB"))/255.)
                X2.append(np.asarray(Image.open(fake_img[random2[0]]).convert("RGB"))/255.)
                y.append(np.array([anchor_choice]))
            else:
                if (len(ft_fake_idx) < batch_size):
                    break
                if (len(real_idx) < batch_size):
                    break
                if (len(fake_idx) < batch_size):
                    break
                ft_fake_idx = ft_fake_idx[~np.isin(ft_fake_idx, anchor_idx)]
                X0.append(np.asarray(Image.open(ft_fake_img[anchor_idx[0]]).convert("RGB"))/255.)
                random1 = np.random.choice(real_idx, 1, replace=False)
                real_idx = real_idx[~np.isin(real_idx, random1)]
                random2 = np.random.choice(fake_idx, 1, replace=False)
                fake_idx = fake_idx[~np.isin(fake_idx, random2)]
                X2.append(np.asarray(Image.open(real_img[random1[0]]).convert("RGB"))/255.)
                X1.append(np.asarray(Image.open(fake_img[random2[0]]).convert("RGB"))/255.)
                y.append(np.array([anchor_choice]))
        X0 = np.asarray(X0)
        X1 = np.asarray(X1)
        X2 = np.asarray(X2)
        y = np.asarray(y)
        yield [X0, X1, X2], y

In [71]:
def manDist(x):
    result = K.exp(-K.sum(K.abs(x[0] - x[1]), axis=1, keepdims=True))
    return result

def euclidean_distance(inputs):
    assert len(inputs) == 2, 'Euclidean distance needs 2 inputs, %d given' % len(inputs)
    x, y = inputs
    return K.sqrt(K.maximum(K.sum(K.square(x - y), axis=1, keepdims=True), K.epsilon())) 

def contrastive_loss(y_true,y_pred):
    margin=1.4
    return K.mean((1. - y_true) * K.square(y_pred) + y_true * K.square(K.maximum(margin - y_pred, 0.)))

def accuracy(y_true, y_pred):
    return K.mean(y_pred[:,0,0] < y_pred[:,1,0])

def y_pred_prt(y_true, y_pred):
    return y_pred

def triplet_loss(y_true, y_pred):
    margin = K.constant(1)
    return K.mean(K.maximum(K.constant(0), K.square(y_pred[:,0,0])-0.5*(K.square(y_pred[:,1,0])+K.square(y_pred[:,2,0])) + margin))

In [72]:
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test50_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(img_height, img_width),
                                                    batch_size=batch_size,
                                                    shuffle=True,
                                                    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(validation_dir,
                                                        target_size=(img_height, img_width),
                                                        batch_size=batch_size,
                                                        shuffle=True,
                                                        class_mode='binary')

test50_generator = test50_datagen.flow_from_directory(test50_dir,
                                                  target_size=(img_height, img_width),
                                                  batch_size=128,
                                                  shuffle=True,
                                                  class_mode='binary')

Found 251702 images belonging to 2 classes.
Found 28298 images belonging to 2 classes.
Found 33086 images belonging to 2 classes.


In [73]:
ft_dir = '/home/skkulab/ICCV/Dataset/preprocessed_dataset/fine-tune_triplet'
train_gen = generator(ft_dir)
test50_gen = generator_res(ft_dir, test50_dir, batch_size=1, critical_value=0.5)

## Modeling

In [10]:
callback_list = [EarlyStopping(monitor='val_acc', patience=5),
                 ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3)]
history = model.fit_generator(train_generator,
                            steps_per_epoch=100,
                            epochs=20,
                            validation_data=validation_generator,
                            validation_steps=len(validation_generator),
                            callbacks=callback_list,
                            verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
100/100 [==============================] - 136s 1s/step - loss: 0.6408 - acc: 0.6188 - val_loss: 0.4574 - val_acc: 0.9462
Epoch 2/20
100/100 [==============================] - 119s 1s/step - loss: 0.2337 - acc: 0.9397 - val_loss: 0.1978 - val_acc: 0.9281
Epoch 3/20
100/100 [==============================] - 118s 1s/step - loss: 0.1343 - acc: 0.9556 - val_loss: 0.1084 - val_acc: 0.9685
Epoch 4/20
100/100 [==============================] - 117s 1s/step - loss: 0.1168 - acc: 0.9609 - val_loss: 0.1214 - val_acc: 0.9618
Epoch 5/20
100/100 [==============================] - 116s 1s/step - loss: 0.0880 - acc: 0.9738 - val_loss: 0.0893 - val_acc: 0.9723
Epoch 6/20
100/100 [==============================] - 116s 1s/step - loss: 0.0967 - acc: 0.9697 - val_loss: 0.0906 - val_acc: 0.9742
Epoch 7/20
100/100 [==============================] - 116s 1s/step - loss: 0.0924 - acc: 0.9756 - val_loss: 0.0878 - val_acc: 0.9728
Epoch 8/20
100/100 [=

In [28]:
test50_classes = test50_generator.classes
print("50% ", len(test50_classes[test50_classes == 1]))

50%  16543


In [12]:
model.save("kalexnet.h5")

### Triplet Modeling Modeling

In [74]:
model = load_model('alexnet.h5')
base_model = Model(img_input, out)
base_model.set_weights(model.get_weights())
for l in range(len(base_model.layers) - 2):
    base_model.layers[l].trainable = False

im_in = Input(shape=(224, 224, 3))
x1 = base_model([im_in])
x1 = Dense(64)(x1)
x1 = Activation('relu')(x1)
x1 = Lambda(lambda x: K.l2_normalize(x, axis=1))(x1)

model_top = Model(inputs=[im_in], outputs=x1)
model_top.summary()

anchor_input = Input(shape=(224, 224, 3))
positive_input = Input(shape=(224, 224, 3))
negative_input = Input(shape=(224, 224, 3))

h_anchor = model_top(anchor_input)
h_positive = model_top(positive_input)
h_negative = model_top(negative_input)

positive_dist = Lambda(euclidean_distance)([h_anchor, h_positive])
negative_dist = Lambda(euclidean_distance)([h_anchor, h_negative])
tertiary_dist = Lambda(euclidean_distance)([h_positive, h_negative])

stacked_dists = Lambda(lambda vects: K.stack(vects, axis=1))([positive_dist, negative_dist, tertiary_dist])
model = Model([anchor_input, positive_input, negative_input], stacked_dists)
model.compile(optimizer=Adam(), loss=tiplet_loss, metrics=[accuracy])
model.summary()

# callback_list = [EarlyStopping(monitor='acc', patience=3)]
output = model.fit_generator(train_gen, steps_per_epoch=40, epochs=10)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_23 (InputLayer)        (None, 224, 224, 3)       0         
_________________________________________________________________
model_18 (Model)             (None, 1)                 4802593   
_________________________________________________________________
dense_10 (Dense)             (None, 64)                128       
_________________________________________________________________
activation_20 (Activation)   (None, 64)                0         
_________________________________________________________________
lambda_26 (Lambda)           (None, 64)                0         
Total params: 4,802,721
Trainable params: 4,225
Non-trainable params: 4,798,496
_________________________________________________________________
__________________________________________________________________________________________________
Layer (type)                 

In [ ]:
## siam model을 통째로 저장할 때 load err: axes doesn't match ##
''' model 설정 후, load weight '''
model_json = model_top.to_json()
with open("siam_alex.json", "w") as json_file:
    json_file.write(model_json)
model_top.save_weights("triplet_alex1.h5")

## Evaluate Model - 50

In [75]:
score50_2 = []
answer50_2 = []
for i in tqdm(test50_gen):
    y_score50_2 = model.predict_on_batch(i[0])
    score50_2.append(y_score50_2)
    answer50_2.append(i[1])
    
score50_2 = np.concatenate(score50_2)
answer50_2 = np.concatenate(answer50_2)


0it [00:00, ?it/s]
1it [00:00,  1.23it/s]
6it [00:00,  1.74it/s]
11it [00:01,  2.44it/s]
16it [00:01,  3.41it/s]
21it [00:01,  4.72it/s]
26it [00:01,  6.45it/s]
31it [00:01,  8.69it/s]
36it [00:01, 11.38it/s]
41it [00:01, 14.60it/s]
46it [00:01, 18.12it/s]
51it [00:01, 21.99it/s]
56it [00:02, 26.15it/s]
61it [00:02, 29.88it/s]
66it [00:02, 32.89it/s]
71it [00:02, 36.18it/s]
76it [00:02, 38.82it/s]
81it [00:02, 40.99it/s]
86it [00:02, 42.35it/s]
91it [00:02, 43.71it/s]
96it [00:02, 44.05it/s]
101it [00:03, 45.04it/s]
106it [00:03, 44.78it/s]
111it [00:03, 43.90it/s]
116it [00:03, 44.77it/s]
121it [00:03, 45.10it/s]
126it [00:03, 44.35it/s]
131it [00:03, 45.01it/s]
136it [00:03, 44.04it/s]
141it [00:03, 43.95it/s]
146it [00:04, 45.13it/s]
151it [00:04, 44.56it/s]
156it [00:04, 45.50it/s]
162it [00:04, 47.32it/s]
167it [00:04, 46.67it/s]
172it [00:04, 46.93it/s]
177it [00:04, 46.40it/s]
182it [00:04, 46.71it/s]
187it [00:04, 46.77it/s]
192it [00:05, 47.18it/s]
197it [00:05, 46.55it/s]
20

1463it [00:32, 45.46it/s]
1469it [00:33, 46.53it/s]
1474it [00:33, 44.72it/s]
1479it [00:33, 45.13it/s]
1484it [00:33, 45.45it/s]
1489it [00:33, 45.17it/s]
1494it [00:33, 45.06it/s]
1499it [00:33, 45.43it/s]
1504it [00:33, 45.34it/s]
1509it [00:33, 45.56it/s]
1514it [00:34, 45.62it/s]
1519it [00:34, 45.87it/s]
1524it [00:34, 46.25it/s]
1529it [00:34, 46.81it/s]
1534it [00:34, 46.33it/s]
1539it [00:34, 46.30it/s]
1544it [00:34, 45.67it/s]
1549it [00:34, 46.28it/s]
1554it [00:34, 46.29it/s]
1559it [00:34, 46.37it/s]
1564it [00:35, 46.70it/s]
1569it [00:35, 46.62it/s]
1574it [00:35, 46.85it/s]
1579it [00:35, 45.95it/s]
1584it [00:35, 45.65it/s]
1589it [00:35, 45.51it/s]
1594it [00:35, 45.98it/s]
1599it [00:35, 46.63it/s]
1604it [00:35, 46.33it/s]
1609it [00:36, 46.19it/s]
1614it [00:36, 45.65it/s]
1619it [00:36, 45.70it/s]
1624it [00:36, 46.63it/s]
1629it [00:36, 45.97it/s]
1634it [00:36, 45.12it/s]
1639it [00:36, 45.19it/s]
1644it [00:36, 44.88it/s]
1649it [00:36, 45.10it/s]
1654it [00:3

2884it [01:04, 46.72it/s]
2889it [01:04, 46.59it/s]
2895it [01:04, 47.46it/s]
2900it [01:04, 46.33it/s]
2905it [01:04, 46.07it/s]
2910it [01:04, 44.69it/s]
2915it [01:04, 45.61it/s]
2920it [01:05, 45.69it/s]
2925it [01:05, 46.03it/s]
2930it [01:05, 44.57it/s]
2935it [01:05, 44.87it/s]
2940it [01:05, 45.38it/s]
2945it [01:05, 46.05it/s]
2950it [01:05, 46.04it/s]
2955it [01:05, 45.89it/s]
2960it [01:05, 46.18it/s]
2965it [01:05, 45.96it/s]
2970it [01:06, 46.06it/s]
2975it [01:06, 45.03it/s]
2980it [01:06, 44.75it/s]
2985it [01:06, 45.51it/s]
2990it [01:06, 46.05it/s]
2995it [01:06, 46.12it/s]
3000it [01:06, 45.68it/s]
3005it [01:06, 46.38it/s]
3010it [01:06, 46.71it/s]
3015it [01:07, 46.05it/s]
3020it [01:07, 45.74it/s]
3025it [01:07, 44.98it/s]
3030it [01:07, 44.86it/s]
3035it [01:07, 45.54it/s]
3040it [01:07, 45.84it/s]
3045it [01:07, 45.41it/s]
3050it [01:07, 45.42it/s]
3055it [01:07, 45.98it/s]
3060it [01:08, 45.67it/s]
3065it [01:08, 45.91it/s]
3070it [01:08, 45.58it/s]
3075it [01:0

4298it [01:35, 47.33it/s]
4303it [01:35, 46.05it/s]
4308it [01:35, 46.37it/s]
4313it [01:35, 46.79it/s]
4318it [01:35, 46.67it/s]
4323it [01:35, 45.53it/s]
4328it [01:35, 45.18it/s]
4333it [01:36, 45.69it/s]
4338it [01:36, 45.42it/s]
4343it [01:36, 44.65it/s]
4348it [01:36, 44.47it/s]
4353it [01:36, 45.59it/s]
4358it [01:36, 44.32it/s]
4363it [01:36, 43.89it/s]
4368it [01:36, 42.91it/s]
4373it [01:37, 43.23it/s]
4378it [01:37, 42.17it/s]
4383it [01:37, 43.42it/s]
4388it [01:37, 43.35it/s]
4393it [01:37, 44.03it/s]
4398it [01:37, 44.60it/s]
4403it [01:37, 44.98it/s]
4408it [01:37, 45.52it/s]
4413it [01:37, 45.22it/s]
4418it [01:38, 44.80it/s]
4423it [01:38, 44.75it/s]
4428it [01:38, 44.98it/s]
4433it [01:38, 45.20it/s]
4438it [01:38, 44.94it/s]
4443it [01:38, 44.77it/s]
4448it [01:38, 45.00it/s]
4453it [01:38, 43.93it/s]
4458it [01:38, 45.01it/s]
4463it [01:39, 45.43it/s]
4468it [01:39, 46.40it/s]
4473it [01:39, 46.36it/s]
4478it [01:39, 47.21it/s]
4483it [01:39, 46.92it/s]
4488it [01:3

5710it [02:06, 45.52it/s]
5715it [02:06, 45.52it/s]
5720it [02:06, 45.48it/s]
5725it [02:06, 45.48it/s]
5730it [02:06, 45.53it/s]
5735it [02:07, 46.39it/s]
5740it [02:07, 46.10it/s]
5745it [02:07, 46.13it/s]
5750it [02:07, 46.80it/s]
5755it [02:07, 46.45it/s]
5760it [02:07, 46.07it/s]
5765it [02:07, 46.28it/s]
5770it [02:07, 46.13it/s]
5775it [02:07, 45.88it/s]
5780it [02:08, 45.81it/s]
5785it [02:08, 45.78it/s]
5790it [02:08, 46.62it/s]
5795it [02:08, 46.71it/s]
5801it [02:08, 48.09it/s]
5807it [02:08, 48.61it/s]
5812it [02:08, 47.65it/s]
5817it [02:08, 47.21it/s]
5822it [02:08, 47.02it/s]
5827it [02:09, 46.24it/s]
5832it [02:09, 46.16it/s]
5837it [02:09, 46.31it/s]
5842it [02:09, 45.88it/s]
5847it [02:09, 44.92it/s]
5852it [02:09, 45.44it/s]
5857it [02:09, 46.04it/s]
5862it [02:09, 46.28it/s]
5867it [02:09, 45.65it/s]
5872it [02:10, 44.96it/s]
5877it [02:10, 45.48it/s]
5882it [02:10, 44.14it/s]
5887it [02:10, 44.51it/s]
5892it [02:10, 43.59it/s]
5897it [02:10, 45.25it/s]
5902it [02:1

...

9962it [03:40, 46.10it/s]
9967it [03:40, 46.53it/s]
9972it [03:40, 45.62it/s]
9977it [03:40, 46.27it/s]
9982it [03:40, 46.88it/s]
9987it [03:40, 46.76it/s]
9992it [03:40, 45.84it/s]
9997it [03:40, 45.35it/s]
10002it [03:41, 45.74it/s]
10007it [03:41, 45.90it/s]
10012it [03:41, 46.21it/s]
10017it [03:41, 46.69it/s]
10022it [03:41, 47.49it/s]
10027it [03:41, 47.24it/s]
10032it [03:41, 46.54it/s]
10037it [03:41, 46.10it/s]
10042it [03:41, 45.58it/s]
10047it [03:42, 45.85it/s]
10052it [03:42, 45.70it/s]
10057it [03:42, 46.22it/s]
10062it [03:42, 44.35it/s]
10067it [03:42, 43.79it/s]
10072it [03:42, 44.60it/s]
10077it [03:42, 45.59it/s]
10082it [03:42, 46.10it/s]
10087it [03:42, 46.13it/s]
10092it [03:43, 45.88it/s]
10097it [03:43, 46.05it/s]
10102it [03:43, 46.69it/s]
10107it [03:43, 46.05it/s]
10112it [03:43, 45.43it/s]
10117it [03:43, 46.08it/s]
10122it [03:43, 46.90it/s]
10127it [03:43, 46.11it/s]
10132it [03:43, 45.68it/s]
10137it [03:44, 46.01it/s]
10142it [03:44, 46.92it/s]
10147it [

11351it [04:10, 48.23it/s]
11356it [04:10, 48.37it/s]
11361it [04:10, 46.95it/s]
11366it [04:10, 47.67it/s]
11372it [04:11, 48.62it/s]
11377it [04:11, 48.26it/s]
11382it [04:11, 48.07it/s]
11387it [04:11, 46.41it/s]
11392it [04:11, 46.37it/s]
11397it [04:11, 45.89it/s]
11402it [04:11, 45.54it/s]
11407it [04:11, 46.37it/s]
11413it [04:11, 47.82it/s]
11418it [04:12, 47.91it/s]
11423it [04:12, 48.01it/s]
11428it [04:12, 46.83it/s]
11434it [04:12, 47.89it/s]
11439it [04:12, 46.33it/s]
11444it [04:12, 46.56it/s]
11449it [04:12, 45.60it/s]
11455it [04:12, 47.47it/s]
11460it [04:12, 46.38it/s]
11465it [04:13, 46.55it/s]
11471it [04:13, 48.32it/s]
11477it [04:13, 50.56it/s]
11483it [04:13, 52.08it/s]
11489it [04:13, 50.03it/s]
11495it [04:13, 50.66it/s]
11501it [04:13, 47.01it/s]
11506it [04:13, 46.97it/s]
11512it [04:13, 48.54it/s]
11517it [04:14, 48.55it/s]
11522it [04:14, 48.48it/s]
11527it [04:14, 47.30it/s]
11532it [04:14, 47.31it/s]
11537it [04:14, 47.54it/s]
11542it [04:14, 46.43it/s]
1

12742it [04:41, 47.48it/s]
12747it [04:41, 46.97it/s]
12752it [04:41, 47.20it/s]
12757it [04:41, 46.58it/s]
12762it [04:41, 46.85it/s]
12767it [04:41, 47.08it/s]
12772it [04:42, 46.18it/s]
12777it [04:42, 45.22it/s]
12782it [04:42, 43.94it/s]
12787it [04:42, 45.47it/s]
12792it [04:42, 46.06it/s]
12797it [04:42, 45.93it/s]
12802it [04:42, 45.71it/s]
12807it [04:42, 45.90it/s]
12812it [04:42, 45.99it/s]
12817it [04:43, 45.85it/s]
12822it [04:43, 45.23it/s]
12827it [04:43, 44.08it/s]
12832it [04:43, 44.63it/s]
12837it [04:43, 45.67it/s]
12842it [04:43, 46.10it/s]
12847it [04:43, 45.40it/s]
12852it [04:43, 45.46it/s]
12857it [04:43, 46.44it/s]
12863it [04:44, 47.80it/s]
12868it [04:44, 48.22it/s]
12873it [04:44, 47.77it/s]
12878it [04:44, 47.96it/s]
12883it [04:44, 46.88it/s]
12888it [04:44, 46.91it/s]
12893it [04:44, 47.02it/s]
12898it [04:44, 46.26it/s]
12903it [04:44, 44.28it/s]
12908it [04:45, 43.73it/s]
12913it [04:45, 43.74it/s]
12918it [04:45, 43.98it/s]
12923it [04:45, 44.57it/s]
1

14113it [05:11, 44.70it/s]
14118it [05:11, 45.08it/s]
14123it [05:11, 43.61it/s]
14128it [05:11, 43.16it/s]
14133it [05:11, 43.82it/s]
14138it [05:11, 41.80it/s]
14143it [05:12, 42.57it/s]
14148it [05:12, 42.01it/s]
14153it [05:12, 41.79it/s]
14158it [05:12, 43.00it/s]
14163it [05:12, 44.15it/s]
14168it [05:12, 44.73it/s]
14174it [05:12, 46.47it/s]
14179it [05:12, 46.65it/s]
14184it [05:12, 46.80it/s]
14190it [05:13, 48.25it/s]
14195it [05:13, 47.12it/s]
14200it [05:13, 47.17it/s]
14205it [05:13, 46.70it/s]
14210it [05:13, 46.71it/s]
14215it [05:13, 46.20it/s]
14220it [05:13, 45.79it/s]
14225it [05:13, 45.71it/s]
14230it [05:13, 44.87it/s]
14235it [05:14, 46.07it/s]
14240it [05:14, 46.48it/s]
14245it [05:14, 46.39it/s]
14250it [05:14, 46.40it/s]
14255it [05:14, 44.80it/s]
14260it [05:14, 44.85it/s]
14265it [05:14, 43.96it/s]
14270it [05:14, 44.45it/s]
14275it [05:14, 43.96it/s]
14280it [05:15, 43.96it/s]
14285it [05:15, 45.02it/s]
14290it [05:15, 45.79it/s]
14295it [05:15, 45.92it/s]
1

15504it [05:41, 46.64it/s]
15509it [05:41, 47.34it/s]
15514it [05:41, 47.01it/s]
15519it [05:41, 47.39it/s]
15524it [05:41, 47.57it/s]
15529it [05:41, 46.12it/s]
15534it [05:41, 46.08it/s]
15539it [05:42, 47.03it/s]
15544it [05:42, 46.14it/s]
15549it [05:42, 45.13it/s]
15554it [05:42, 46.13it/s]
15559it [05:42, 45.08it/s]
15564it [05:42, 45.24it/s]
15569it [05:42, 45.87it/s]
15574it [05:42, 46.67it/s]
15579it [05:42, 46.50it/s]
15584it [05:42, 46.66it/s]
15589it [05:43, 45.61it/s]
15594it [05:43, 45.84it/s]
15599it [05:43, 45.68it/s]
15604it [05:43, 45.38it/s]
15609it [05:43, 46.59it/s]
15614it [05:43, 45.89it/s]
15619it [05:43, 46.38it/s]
15624it [05:43, 46.70it/s]
15629it [05:43, 46.94it/s]
15634it [05:44, 46.91it/s]
15639it [05:44, 46.53it/s]
15644it [05:44, 44.63it/s]
15649it [05:44, 44.86it/s]
15654it [05:44, 45.55it/s]
15659it [05:44, 44.51it/s]
15664it [05:44, 45.22it/s]
15669it [05:44, 45.28it/s]
15674it [05:44, 45.70it/s]
15679it [05:45, 46.48it/s]
15684it [05:45, 46.68it/s]
1

In [80]:
np.mean(score50_2[:,0,0] < score50_2[:,1,0])

0.9894819561143686

In [83]:
y_hat50_2 = np.zeros_like(answer50_2)
for i in range(len(score50_2)):
    if score50_2[i][0, 0] < score50_2[i][1, 0]:
        y_hat50_2[i] = answer50_2[i]
    else:
        if answer50_2[i] == 0:
            y_hat50_2[i] = 1
        else:
            y_hat50_2[i] = 0

In [84]:
print(metrics.classification_report(answer50_2, y_hat50_2))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      8280
           1       1.00      0.98      0.99      8263

    accuracy                           0.99     16543
   macro avg       0.99      0.99      0.99     16543
weighted avg       0.99      0.99      0.99     16543

